In [26]:
from dotenv import load_dotenv
import jikanpy
import os
import requests
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
load_dotenv()

secret = os.getenv("secret")
id = os.getenv("id")
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_KEY")
jikan = jikanpy.Jikan()

In [2]:
from boto3.dynamodb.types import TypeDeserializer
import boto3

def get_analytics_data():
    # Connect to dynamodb
    dynamodb = boto3.client(
        "dynamodb",
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        region_name="us-west-2"
    )
    df_dicts = {}
    
    
    # get top 25 data
    response_top_25 = dynamodb.scan(TableName="anime-dashboard-top-25",)
    
    # convert to pandas dataframe
    df_dicts['top_25'] = parse_analytics_data(response_top_25['Items'])
    
    
    return df_dicts


def _deserialize(data):
    deserializer = TypeDeserializer()
    return {k: deserializer.deserialize(v) for k, v in data.items()}

def parse_analytics_data(data):
    deserialized_data = [_deserialize(x) for x in data]
    data = [pd.json_normalize(x, 'data', ['timestamp']) for x in deserialized_data]
    return pd.concat(data, ignore_index=True)

In [3]:
data = get_analytics_data()
df = data['top_25']
df

,title_japanese,image_url,genres,title_english,mal_id,title,url,studios.urls,studios.names,licensors.urls,licensors.names,statistics.favorites,statistics.scored_by,statistics.rank,statistics.score,statistics.members,statistics.popularity,timestamp
0,葬送のフリーレン,https://cdn.myanimelist.net/images/anime/1015/...,"[Adventure, Drama, Fantasy]",Frieren: Beyond Journey's End,52991,Sousou no Frieren,https://myanimelist.net/anime/52991/Sousou_no_...,[https://myanimelist.net/anime/producer/11/Mad...,[Madhouse],[https://myanimelist.net/anime/producer/1468/C...,[Crunchyroll],61056,583970,1,9.31,1007399,166,2025-01-27T02:26:56Z
1,鋼の錬金術師 FULLMETAL ALCHEMIST,https://cdn.myanimelist.net/images/anime/1208/...,"[Action, Adventure, Drama, Fantasy]",Fullmetal Alchemist: Brotherhood,5114,Fullmetal Alchemist: Brotherhood,https://myanimelist.net/anime/5114/Fullmetal_A...,[https://myanimelist.net/anime/producer/4/Bones],[Bones],[https://myanimelist.net/anime/producer/102/Fu...,"[Funimation, Aniplex of America]",231137,2187771,2,9.1,3468468,3,2025-01-27T02:26:56Z
2,ONE PIECE FAN LETTER,https://cdn.myanimelist.net/images/anime/1455/...,"[Action, Adventure, Fantasy]",None,60022,One Piece Fan Letter,https://myanimelist.net/anime/60022/One_Piece_...,[https://myanimelist.net/anime/producer/18/Toe...,[Toei Animation],[],[],1906,63986,3,9.07,84069,2462,2025-01-27T02:26:56Z
3,STEINS;GATE,https://cdn.myanimelist.net/images/anime/1935/...,"[Drama, Sci-Fi, Suspense]",Steins;Gate,9253,Steins;Gate,https://myanimelist.net/anime/9253/Steins_Gate,[https://myanimelist.net/anime/producer/314/Wh...,[White Fox],[https://myanimelist.net/anime/producer/102/Fu...,[Funimation],193789,1444184,4,9.07,2656952,14,2025-01-27T02:26:56Z
4,進撃の巨人 Season3 Part.2,https://cdn.myanimelist.net/images/anime/1517/...,"[Action, Drama, Suspense]",Attack on Titan Season 3 Part 2,38524,Shingeki no Kyojin Season 3 Part 2,https://myanimelist.net/anime/38524/Shingeki_n...,[https://myanimelist.net/anime/producer/858/Wi...,[Wit Studio],[https://myanimelist.net/anime/producer/102/Fu...,[Funimation],60298,1661812,5,9.05,2394326,21,2025-01-27T02:26:56Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1845,劇場版 銀魂 完結篇 万事屋よ永遠なれ,https://cdn.myanimelist.net/images/anime/10/51...,"[Action, Comedy, Sci-Fi]",Gintama: The Movie: The Final Chapter: Be Fore...,15335,Gintama Movie 2: Kanketsu-hen - Yorozuya yo Ei...,https://myanimelist.net/anime/15335/Gintama_Mo...,[https://myanimelist.net/anime/producer/14/Sun...,[Sunrise],[],[],2123,138846,21,8.9,248145,1031,2025-01-26T03:56:56Z
1846,薬屋のひとりごと,https://cdn.myanimelist.net/images/anime/1708/...,"[Drama, Mystery]",The Apothecary Diaries,54492,Kusuriya no Hitorigoto,https://myanimelist.net/anime/54492/Kusuriya_n...,[https://myanimelist.net/anime/producer/28/OLM...,"[OLM, TOHO animation STUDIO]",[https://myanimelist.net/anime/producer/1468/C...,[Crunchyroll],15930,282689,23,8.88,543639,432,2025-01-26T03:56:56Z
1847,銀魂. 銀ノ魂篇 後半戦,https://cdn.myanimelist.net/images/anime/1776/...,"[Action, Comedy, Sci-Fi]",Gintama. Silver Soul Arc - Second Half War,37491,Gintama. Shirogane no Tamashii-hen - Kouhan-sen,https://myanimelist.net/anime/37491/Gintama_Sh...,[https://myanimelist.net/anime/producer/1258/B...,[Bandai Namco Pictures],[],[],1062,100008,22,8.88,208331,1229,2025-01-26T03:56:56Z
1848,モンスター,https://cdn.myanimelist.net/images/anime/10/18...,"[Drama, Mystery, Suspense]",Monster,19,Monster,https://myanimelist.net/anime/19/Monster,[https://myanimelist.net/anime/producer/11/Mad...,[Madhouse],[https://myanimelist.net/anime/producer/119/VI...,[VIZ Media],56275,453619,24,8.88,1198905,126,2025-01-26T03:56:56Z


In [4]:
def members_line_plot(data):
    """
    Function to create a line plot of the differences in members data over time,
    showing the most recent day and allowing scrolling back for the past 100 days.
    """

    # Ensure timestamp is in datetime format
    data['timestamp'] = pd.to_datetime(data['timestamp'])

    # Calculate the difference in 'statistics.members' for each anime
    data['members_diff'] = data.groupby('title')['statistics.members'].diff()

    # Sort by timestamp
    data.sort_values(by='timestamp', inplace=True)

    # Find the most recent timestamp and calculate the 100-day range
    most_recent_date = data['timestamp'].max()
    start_date = most_recent_date - pd.Timedelta(days=1)

    # Create the Plotly line plot
    fig = px.line(
        data,
        x='timestamp',
        y='members_diff',
        color='title',
        title="Change in Members Over Time by Anime",
        labels={
            "timestamp": "Timestamp",
            "members_diff": "Change in Members",
            "title": "Anime Title"
        },
    )

    # Update the x-axis range and add grid lines
    fig.update_layout(
        xaxis=dict(
            range=[start_date, most_recent_date],
            showgrid=True,
            rangeslider=dict(visible=True)  # Enable range slider for easier navigation
        ),
        yaxis=dict(showgrid=True),
    )

    return fig


In [5]:
members_line_plot(df.copy())

In [6]:
import pandas as pd
import plotly.express as px

def studios_in_top_25(data):
    """
    Function to create a bar chart to switch between displaying studio and licensor data,
    with hover information showing associated titles and a button to toggle between the two.
    """
    # Get most recent date
    most_recent_date = data['timestamp'].max()
    data = data[data['timestamp'] == most_recent_date]

    # Prepare the studio data
    studio_data = data[['title', 'studios.names']].dropna()
    studio_data = studio_data.explode('studios.names')  # Explode nested list into rows
    studio_data = (
        studio_data.groupby('studios.names')
        .agg(titles=('title', list), count=('studios.names', 'count'))
        .reset_index()
        .rename(columns={'studios.names': 'name'})
    )
    studio_data['type'] = 'Studio'

    # Prepare the licensor data
    licensor_data = data[['title', 'licensors.names']].dropna()
    licensor_data = licensor_data.explode('licensors.names')  # Explode nested list into rows
    licensor_data = (
        licensor_data.groupby('licensors.names')
        .agg(titles=('title', list), count=('licensors.names', 'count'))
        .reset_index()
        .rename(columns={'licensors.names': 'name'})
    )
    licensor_data['type'] = 'Licensor'

    # Combine the data
    combined_data = pd.concat([studio_data, licensor_data], ignore_index=True)

    # Add a hover data for titles
    combined_data['hover_titles'] = combined_data['titles'].apply(lambda x: ', '.join(x))

    # Create the bar chart
    fig = px.bar(
        combined_data,
        x='name',
        y='count',
        color='type',
        hover_data={'hover_titles': True, 'count': True, 'type': False},
        title="Number of Studios vs Licensors",
        labels={
            'name': 'Name',
            'count': 'Number of Titles',
        }
    )

    # Update the layout for better appearance and add toggle buttons
    fig.update_layout(
        xaxis=dict(title="Name", tickangle=45),
        yaxis=dict(title="Number of Titles"),
        updatemenus=[
            {
                "buttons": [
                    {
                        "label": "Studio",
                        "method": "update",
                        "args": [
                            {"visible": [True, False]},  # Show only Studio bars
                            {"title": "Number of Studios"}
                        ],
                    },
                    {
                        "label": "Licensor",
                        "method": "update",
                        "args": [
                            {"visible": [False, True]},  # Show only Licensor bars
                            {"title": "Number of Licensors"}
                        ],
                    },
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 10},
                "showactive": True,
                "type": "buttons",
                "x": 0.1,
                "xanchor": "left",
                "y": 1.2,
                "yanchor": "top",
            }
        ]
    )

    return fig


In [7]:
studios_in_top_25(df.copy())

In [37]:
import warnings
warnings.filterwarnings("ignore")

In [80]:
import numpy as np
import plotly.graph_objects as go

# Get the most recent timestamp
most_recent_date = df['timestamp'].max()
df_recent = df[df['timestamp'] == most_recent_date]

# Ensure numeric conversion
df_recent['statistics.popularity'] = pd.to_numeric(df_recent['statistics.popularity'], errors='coerce')
df_recent['statistics.score'] = pd.to_numeric(df_recent['statistics.score'], errors='coerce')
df_recent['statistics.members'] = pd.to_numeric(df_recent['statistics.members'], errors='coerce')

# Normalize and invert 'statistics.popularity'
df_recent['inverted_popularity'] = 1 / (df_recent['statistics.popularity'] + 1)  # Avoid division by zero

# Apply logarithmic scaling for marker size
df_recent['log_scaled_size'] = np.log1p(df_recent['inverted_popularity'])  # log1p to avoid log(0)
df_recent['log_scaled_size'] = (
    (df_recent['log_scaled_size'] - df_recent['log_scaled_size'].min()) /
    (df_recent['log_scaled_size'].max() - df_recent['log_scaled_size'].min())
) * (60 - 15) + 15  # Scale between min (15) and max (60)

# Create the scatter plot using go
fig = go.Figure()

for i, row in df_recent.iterrows():
    fig.add_trace(
        go.Scatter(
            x=[row['statistics.members']],
            y=[row['statistics.score']],
            mode='markers',
            marker=dict(
                size=row['log_scaled_size'],  # Use the log-scaled size
                sizemode='area',
                color=row['statistics.score'],  # Color based on score
                showscale=True,
            ),
            name=row['title_english'],  # Use title for legend
            text=row['title_english'],  # Use title for hover text
        )
    )

# Update the layout for better appearance
fig.update_layout(
    title="Anime Popularity vs. Score by Members (Log-Scaled Sizes)",
    xaxis=dict(title="Members", type="log"),  # Log scale for x-axis
    yaxis=dict(title="Score"),
    showlegend=False,
)

fig.show()
